<a href="https://colab.research.google.com/github/yyyyawend/Checkers-AI/blob/master/5_Multipletest_Data_Organization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/Life_Singularity')

Mounted at /content/drive


In [ ]:
!pip install PyRuSH
!wget https://github.com/jianlins/PyRuSH/raw/master/conf/rush_rules.tsv -P conf
!pip install wget

In [ ]:
#use PyRush to split text into sentences
from PyRuSH import RuSH
rush = RuSH('conf/rush_rules.tsv')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
BRANCH="main"

In [ ]:
#restart the runtime after running this cell in order to use newly installed versions.
!apt-get update && apt-get install -y libsndfile1 ffmpeg
!pip install Cython
!pip install nemo_toolkit['all']
!apt-get install cuda-11-6

In [ ]:
!git clone https://github.com/ericharper/apex.git
!cd apex
!git checkout nm_v1.13.0
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" --global-option="--fast_layer_norm" --global-option="--distributed_adam" --global-option="--deprecated_fused_adam" /content/apex/

In [ ]:
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager

import os
import wget

import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[NeMo W 2022-12-27 04:34:10 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-12-27 04:34:11 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-12-27 04:34:13 nemo_logging:349] /usr/local/lib/python3.8/dist-packages/torch/jit/annotations.py:299: UserWarning: TorchScript will treat type annotations of Tensor dtype-specific subtypes as if they are normal Tensors. dtype constraints are not enforced in compilation either.
      warnings.warn("TorchScript will treat type annotations of Tensor "
    


# Task Description

In this task, we will using the fine tuned NER model get the entity class of each token, and the relation between entity pairs by the fine tuned RE model. Then Organize NER and RE output to a sqlite database so that we can select and plot information need.

# Processing Data


In [ ]:
import os
DATA_DIR = "/content/drive/MyDrive/Life_Singularity/Multiple_Test_Extraction/NER/DATA_DIR/"
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, 'NER'), exist_ok=True)
NER_DATA_DIR = f'{DATA_DIR}/NER'

In [ ]:
#read raw data
with open("/content/drive/MyDrive/Life_Singularity/DATADIR/Second file - Greater than 1000 Patients - Notevents.csv", 'r') as f:
    read_data = f.readlines()

with open("/content/drive/MyDrive/Life_Singularity/DATADIR/Greater_than_1000 Patients-Notevents.txt", 'r') as f:
    read_data1 = f.readlines()

data = read_data + read_data1

In [ ]:
import pandas as pd

In [ ]:
#check patients have more than 3 admission
import re
line_pattern = re.compile(r"(\d*,\d*,\d*,\d{4}-\d{1,2}-\d{1,2}.*)")
count = {}
for i in range(len(data)):
  if line_pattern.match(data[i]):
    pid = data[i].split(",")[1]
    if pid in count.keys():
      count[pid] += 1
    else:
      count[pid] = 1

pati_multiple_admi = []
for k, v in count.items():
  if v >= 3:
   pati_multiple_admi.append(k)

In [ ]:
#Separate patients by the first line of each patient
data1 = ''.join(data)
pattern = r"(\d*,\d*,\d*,\d{4}-\d{1,2}-\d{1,2}.*)" # 174,22532,167853,2151-08-04 ****
input = re.split(pattern, data1)
del input[0]               #del "" at the beginning

In [ ]:
#Link first line and the remaining content of each patient
pers = {}
notes = []
for i in range(0, len(input), 2):
  pid = input[i].split(',')[1]
  if pid in pati_multiple_admi:
    if pid in pers.keys():
      pers[pid].append(input[i] + input[i + 1])
    else:
      pers[pid] = [input[i] + input[i + 1]]
    notes.append(input[i] + input[i + 1])
numPer = len(pers)

In [ ]:
#check patient with heart disease history:
pati_with_chf = []
for k,v in pers.items():
  for note in v:
    if len(re.findall(r'heart failure', note.lower())) > 0 or len(re.findall('chf', note.lower())) > 0:
      pati_with_chf.append(k)

In [ ]:
#Link first line and the remaining content of each patient
pers = {}
notes = []
for i in range(0, len(input), 2):
  pid = input[i].split(',')[1]
  if pid in pati_multiple_admi:
    if pid in pers.keys():
      pers[pid].append(input[i] + input[i + 1])
    else:
      pers[pid] = [input[i] + input[i + 1]]
    notes.append(input[i] + input[i + 1])
numPer = len(pers)

In [ ]:
import re
def clean(sen):
  """
    clean str:
    1)remove unnecessary sign like '\[\*\*', '\*\*\]' 
    2)convert test_name-test_value to test_name test_value 
    3)seperate date or value with words 
    4)add units to values
   
    :param sen: string need to be cleaned
    :return: cleaned string 
  """ 
  sen = sen.lower()    #convert all characters to lower case
 
  #remove "-" between test name and test value. eg. 'creat-2.0' to 'creat 2.0'
  sen = re.sub(r"([^\d]+)-(\d+\.{0,1}\d{1,2})", r"\1 \2", sen)  
  sen = re.sub(r"hco3-(\d+\.{0,1}\d+)", r"hco3 \1 mmol/l", sen) 
  sen = re.sub(r"co2-(\d+\.{0,1}\d+)", r"co2 \1", sen)
   
  sen = re.sub(r"na\+", r"na", sen)  #replace 'na+' with 'na'
  sen = re.sub(r"k\+", r"k", sen) #replace 'k+' with 'k'
  sen = sen.replace("[**", " ") #remove [**
  sen = sen.replace("**]", " ") #remove **]
  sen = sen.replace("*", " ") #remove *

  #add units after test value
  sen = re.sub(r"creat (\d{0,1}\.{0,1}\d+)", r"creat \1 mg/d", sen) 
  sen = re.sub(r"creatinine (\d{0,1}\.{0,1}\d+)", r"creatinine \1 mg/d", sen)
  sen = re.sub(r"creatinine was (\d{0,1}\.{0,1}\d+)", r"creatinine was \1 mg/d", sen)
  sen = re.sub(r"creatinine of (\d{0,1}\.{0,1}\d+)", r"creatinine of \1 mg/d", sen)
  sen = re.sub(r"albumin (\d+\.{0,1}\d+)", r"albumin \1 g/d", sen)
  sen = re.sub(r"totbili (\d+\.{0,1}\d+)", r"totbili \1 mg/d", sen)
  sen = re.sub(r"tot bili (\d+\.{0,1}\d+)", r"totbili \1 mg/d", sen)
  sen = re.sub(r"crp (\d+\.{0,1}\d+)", r"crp \1 mg/l", sen)
  sen = re.sub(r"chloride (\d+\.{0,1}\d+)", r"chloride \1 mmol/l", sen)
  sen = re.sub(r"cl (\d+\.{0,1}\d+)", r"cl \1 mmol/l", sen)
  sen = re.sub(r"hemoglobin (\d+\.{0,1}\d+)", r"hemoglobin \1 g/d", sen)
  sen = re.sub(r"hgb (\d+\.{0,1}\d+)", r"hgb \1 g/dl", sen)
  sen = re.sub(r"lactate (\d+\.{0,1}\d+)", r"lactate \1 mg/d", sen)
  sen = re.sub(r"ld(ldh) (\d+\.{0,1}\d+)", r"ld(ldh) \1 mg/d", sen)
  sen = re.sub(r"mch (\d+\.{0,1}\d+)", r"mch \1 pg/cell", sen)
  sen = re.sub(r"mchc (\d+\.{0,1}\d+)", r"mchc \1 g/d", sen)
  sen = re.sub(r"phosphate (\d+\.{0,1}\d+)", r"phosphate \1 mg/d", sen)
  sen = re.sub(r"phos (\d+\.{0,1}\d+)", r"phos \1 mg/d", sen)
  sen = re.sub(r"potassium (\d+\.{0,1}\d+)", r"potassium \1 mmol/l", sen)
  sen = re.sub(r"k (\d+\.{0,1}\d+)", r"k \1 mmol/l", sen)
  sen = re.sub(r"sodium (\d+\.{0,1}\d+)", r"sodium \1 mmol/l", sen)
  sen = re.sub(r"na (\d+\.{0,1}\d+)", r"na \1 mmol/l", sen)
  sen = re.sub(r"urean (\d+\.{0,1}\d+)", r"urean \1 mg/d", sen)
  sen = re.sub(r"urea n (\d+\.{0,1}\d+)", r"urean \1 mg/d", sen)
  sen = re.sub(r"bun (\d+\.{0,1}\d+)", r"bun \1 mg/d", sen)
  
  #split date and words. 
  sen = re.sub(r"(\d{4}-\d{1,2}-\d{1,2})(.)", r"\1 \2", sen)    #eg. '2102-1-2lab' to '2102-1-2 lab'
  sen = re.sub(r" ((0[1-9]|[1-9][^0-9]|1[0-2])-(0[1-9]|[1-9][^0-9]|1[0-9]|2[0-9]|3[0-1]))([^0-9].)", r" \1 \4", sen) #eg. '1-2lab' to '1-2 lab'

  sen = re.sub(r"(\d+[^,-]\.\d{1,2})([a-z]+)", r"\1 \2", sen)  #split value and words. eg. '1.3creat' to '1.3 creat'
  sen = re.sub(r"([f | m])service", r"\1 service", sen) #add space betwee 'sex : f' and 'service'
  sen = sen.replace("\n", " ") #remove "\n"
  sen = re.sub('\,\,+', ',', sen) #replace repeated ',' with one ','
  sen = ' '.join(sen.split())  # replace multiple spaces with a single space
  return sen

In [ ]:
process_data = []
for i in range(len(notes)):
  # split text to sentences
  per_sen = []
  sentences = rush.segToSentenceSpans(notes[i])
  for sen in sentences:
      temp = notes[i][sen.begin:sen.end]
      temp = clean(temp)
      
      temp = word_tokenize(temp)  # tokenize words
      temp = " ".join(temp)
    
      for line in temp.split("newline"):
        if len(line) > 480:
         line = line[:480]
        per_sen.append(line)
  process_data.append(per_sen)

# Implemet Database sqlite3

In [ ]:
import sqlite3
 
conn = sqlite3.connect('/content/drive/MyDrive/Life_Singularity/Multiple_Test_Extraction/result.db')

In [ ]:
#create tables, uncomment lines below if you want to create a new database
# conn.execute('''
# CREATE TABLE Patient(
#    patient_id Integer PRIMARY KEY,
#    subject_id varchar(20) UNIQUE,
#    date_of_birth varchar(20),
#    gender varchar(20)
#  );''')

# conn.execute('''
# CREATE TABLE Admission(
#    admission_id Integer PRIMARY KEY,
#    Hadm varchar(20),
#    admission_date varchar(20),
#    discharge_date varchar(20),
#    chf_diagnosis int
#  );''')

# conn.execute(''' CREATE TABLE Patient_Admission(
#    admission_id int,
#    patient_id int,
#    CONSTRAINT fk_admission_id FOREIGN KEY(admission_id) REFERENCES ADMISSION(admission_id),
#    CONSTRAINT fk_patient_id FOREIGN KEY(patient_id) REFERENCES Patient(patient_id)
#  ); ''')

# conn.execute('''CREATE TABLE Test(
#    test_id Integer PRIMARY KEY AUTOINCREMENT,
#    test_date varchar(50),
#    test_time varchar(20),
#    test_value varchart(20),
#    test_type varchar(50)); ''')

# conn.execute('''CREATE TABLE Admission_Test(
#    admission_id int,
#    test_id int,
#    PRIMARY KEY (admission_id, test_id),
#    CONSTRAINT fk_admission_id FOREIGN KEY(admission_id) REFERENCES ADMISSION(admission_id),
#    CONSTRAINT fk_test_id FOREIGN KEY(test_id) REFERENCES TEST(test_id)
#  );''')

# conn.commit()
 

# Obtained And Process NER Output


**NER Model configuration**

Our Named Entity Recognition model is comprised of the pretrained [BERT](https://arxiv.org/pdf/1810.04805.pdf) model followed by a Token Classification layer.

The model is defined in a config file which declares multiple important sections. They are:
- **model**: All arguments that are related to the Model - language model, token classifier, optimizer and schedulers, datasets and any other related information

- **trainer**: Any argument to be passed to PyTorch Lightning

In [ ]:
NER_MODEL_CONFIG = "token_classification_config.yaml"
NER_WORK_DIR = "/content/drive/MyDrive/Life_Singularity/Multiple_Test_Extraction/NER/WORK_DIR"
os.makedirs(NER_WORK_DIR, exist_ok=True)

In [ ]:
# download the model's configuration file 
NER_config_dir = NER_WORK_DIR + '/configs/'
os.makedirs(NER_config_dir, exist_ok=True)
if not os.path.exists(NER_config_dir + NER_MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/conf/' + NER_MODEL_CONFIG, NER_config_dir)
else:
    print ('config file is already exists')

config file is already exists


In [ ]:
# this line will print the entire config of the model
NER_config_path = f'{NER_WORK_DIR}/configs/{NER_MODEL_CONFIG}'
print(NER_config_path)
NER_config = OmegaConf.load(NER_config_path)
# Note: these are small batch-sizes - increase as appropriate to available GPU capacity
NER_config.model.train_ds.batch_size=20
NER_config.model.validation_ds.batch_size=20

/content/drive/MyDrive/Life_Singularity/Multiple_Test_Extraction/NER/WORK_DIR/configs/token_classification_config.yaml


In [ ]:
print(OmegaConf.to_yaml(NER_config))

**NER Model Training**

Setting up Data within the config

Among other things, the config file contains dictionaries called dataset, train_ds and validation_ds. These are configurations used to setup the Dataset and DataLoaders of the corresponding config.



We assume that both training and evaluation files are located in the same directory, and use the default names mentioned during the data download step. 
So, to start model training, we simply need to specify `model.dataset.data_dir`, like we are going to do below.



Also notice that some config lines, including `model.dataset.data_dir`, have `???` in place of paths, this means that values for these fields are required to be specified by the user.

Let's now add the data directory path to the config.

In [ ]:
# in this tutorial train and dev datasets are located in the same folder, so it is enought to add the path of the data directory to the config
NER_config.model.dataset.data_dir = NER_DATA_DIR
NER_config.model.dataset.max_seq_length = 256

# if you want to decrease the size of your datasets, uncomment the lines below:
# NUM_SAMPLES = 100000
# config.model.train_ds.num_samples = NUM_SAMPLES
# config.model.validation_ds.num_samples = NUM_SAMPLES

**Building the PyTorch Lightning Trainer**

NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem.

Let's first instantiate a Trainer object

In [ ]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(NER_config.trainer))

Trainer config - 

devices: 1
num_nodes: 1
max_epochs: 5
max_steps: -1
accumulate_grad_batches: 1
gradient_clip_val: 0.0
precision: 16
accelerator: gpu
enable_checkpointing: false
logger: false
log_every_n_steps: 1
val_check_interval: 1.0
resume_from_checkpoint: null



In [ ]:
# lets modify some trainer configs
# checks if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
NER_config.trainer.devices = 1
NER_config.trainer.accelerator = accelerator
NER_config.trainer.max_epochs = 3
NER_config.trainer.max_steps = -1
# for PyTorch Native AMP set precision=16
NER_config.trainer.precision = 16 if torch.cuda.is_available() else 32

# remove distributed training flags
NER_config.trainer.strategy = None

NER_trainer = pl.Trainer(**NER_config.trainer)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


**Setting up a NeMo Experiment**

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it:

In [ ]:
NER_exp_dir = exp_manager(NER_trainer, NER_config.get("exp_manager", None))
os.makedirs(NER_WORK_DIR, exist_ok=True)

# the exp_dir provides a path to the current experiment for easy access
NER_exp_dir = str(NER_exp_dir)
NER_exp_dir

[NeMo I 2022-12-03 16:37:33 exp_manager:291] Experiments will be logged at /content/nemo_experiments/token_classification_model/2022-12-03_16-37-33
[NeMo I 2022-12-03 16:37:33 exp_manager:669] TensorboardLogger has been set up


[NeMo W 2022-12-03 16:37:33 nemo_logging:349] /usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:2274: LightningDeprecationWarning: `Trainer.weights_save_path` has been deprecated in v1.6 and will be removed in v1.8.
      rank_zero_deprecation("`Trainer.weights_save_path` has been deprecated in v1.6 and will be removed in v1.8.")
    
[NeMo W 2022-12-03 16:37:33 exp_manager:919] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to -1. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.


'/content/nemo_experiments/token_classification_model/2022-12-03_16-37-33'

To load the pretrained BERT LM model, we can get the list of names by following command 

In [ ]:
from nemo.collections.nlp.models.language_modeling.megatron_bert_model import MegatronBertModel
print([model.pretrained_model_name for model in MegatronBertModel.list_available_models()])

['megatron_bert_345m_cased', 'megatron_bert_345m_uncased', 'biomegatron345m_biovocab_50k_cased', 'biomegatron345m_biovocab_50k_uncased', 'biomegatron345m_biovocab_30k_cased', 'biomegatron345m_biovocab_30k_uncased', 'biomegatron-bert-345m-cased', 'biomegatron-bert-345m-uncased']


In [ ]:
# add the specified above model parameters to the config
# config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL
NER_config.model.language_model.lm_checkpoint = None
NER_config.model.language_model.pretrained_model_name = 'biomegatron345m_biovocab_30k_cased'
NER_config.model.tokenizer.tokenizer_name = None

In [ ]:
#restore NER model
NER_model = nemo_nlp.models.TokenClassificationModel.restore_from(restore_path='/content/drive/My Drive/Life_Singularity/Multiple_Test_Extraction/NER/MODEL/ner_model.nemo', trainer = NER_trainer)

[NeMo W 2022-12-03 16:37:52 lm_utils:80] biomegatron345m_biovocab_30k_cased is not in get_pretrained_lm_models_list(include_external=False), will be using AutoModel from HuggingFace.


[NeMo I 2022-12-03 16:37:52 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/biomegatron345m_biovocab_30k_cased/versions/1/files/BioMegatron345m-biovocab-30k-cased.nemo to /root/.cache/torch/NeMo/NeMo_1.12.0/BioMegatron345m-biovocab-30k-cased/5885010653185bba59bf489ff757bf09/BioMegatron345m-biovocab-30k-cased.nemo
[NeMo I 2022-12-03 16:38:28 common:910] Instantiating model from pre-trained checkpoint
[NeMo I 2022-12-03 16:38:38 megatron_init:204] Rank 0 has data parallel group: [0]
[NeMo I 2022-12-03 16:38:38 megatron_init:207] All data parallel group ranks: [[0]]
[NeMo I 2022-12-03 16:38:38 megatron_init:208] Ranks 0 has data parallel rank: 0
[NeMo I 2022-12-03 16:38:38 megatron_init:216] Rank 0 has model parallel group: [0]
[NeMo I 2022-12-03 16:38:38 megatron_init:217] All model parallel group ranks: [[0]]
[NeMo I 2022-12-03 16:38:38 megatron_init:227] Rank 0 has tensor model parallel group: [0]
[NeMo I 2022-12-03 16:38:38 megatron_init:231] All tensor mo

[NeMo W 2022-12-03 16:38:38 modelPT:217] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.


[NeMo I 2022-12-03 16:38:38 tokenizer_utils:204] Getting Megatron tokenizer for pretrained model name: megatron-bert-345m-cased, custom vocab file: /tmp/tmpgd1unz0e/f67afcc805164750a9eb2aa564aaf9a9_pubmed_merged-all-cased.vocab.txt, and merges file: None
[NeMo I 2022-12-03 16:38:38 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-large-cased, vocab_file: /tmp/tmpgd1unz0e/f67afcc805164750a9eb2aa564aaf9a9_pubmed_merged-all-cased.vocab.txt, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.


...

[NeMo I 2022-12-03 16:39:16 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.12.0/BioMegatron345m-biovocab-30k-cased/5885010653185bba59bf489ff757bf09/BioMegatron345m-biovocab-30k-cased.nemo.
[NeMo I 2022-12-03 16:39:16 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.12.0/BioMegatron345m-biovocab-30k-cased/5885010653185bba59bf489ff757bf09/BioMegatron345m-biovocab-30k-cased.nemo
[NeMo I 2022-12-03 16:39:16 common:910] Instantiating model from pre-trained checkpoint
[NeMo I 2022-12-03 16:39:25 megatron_init:204] Rank 0 has data parallel group: [0]
[NeMo I 2022-12-03 16:39:25 megatron_init:207] All data parallel group ranks: [[0]]
[NeMo I 2022-12-03 16:39:25 megatron_init:208] Ranks 0 has data parallel rank: 0
[NeMo I 2022-12-03 16:39:25 megatron_init:216] Rank 0 has model parallel group: [0]
[NeMo I 2022-12-03 16:39:25 megatron_init:217] All model parallel group ranks: [[0]]
[NeMo I 2022-12-03 16:39:25 megatron_init:227] Rank 0 has tensor model parallel group: [

[NeMo W 2022-12-03 16:39:25 modelPT:217] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.


[NeMo I 2022-12-03 16:39:25 tokenizer_utils:204] Getting Megatron tokenizer for pretrained model name: megatron-bert-345m-cased, custom vocab file: /tmp/tmprav1h29z/f67afcc805164750a9eb2aa564aaf9a9_pubmed_merged-all-cased.vocab.txt, and merges file: None
[NeMo I 2022-12-03 16:39:25 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-large-cased, vocab_file: /tmp/tmprav1h29z/f67afcc805164750a9eb2aa564aaf9a9_pubmed_merged-all-cased.vocab.txt, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.


[NeMo I 2022-12-03 16:39:26 megatron_base_model:185] Padded vocab_size: 31104, original vocab_size: 31079, dummy tokens: 25.
[NeMo I 2022-12-03 16:39:28 save_restore_connector:243] Model MegatronBertModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.12.0/BioMegatron345m-biovocab-30k-cased/5885010653185bba59bf489ff757bf09/BioMegatron345m-biovocab-30k-cased.nemo.
[NeMo I 2022-12-03 16:39:28 nlp_model:174] Registering MegatronBERT model config for biomegatron345m_biovocab_30k_cased is not yet supported.                         Please override this method if needed.
[NeMo I 2022-12-03 16:39:34 save_restore_connector:243] Model TokenClassificationModel was successfully restored from /content/drive/My Drive/Life_Singularity/Multiple_Test_Extraction/NER/MODEL/ner_model.nemo.


In [ ]:
NER_model.half()

# RE Model configuration

Now, let's take a closer look at the model's configuration and learn to train the model.

The model is defined in a config file which declares multiple important sections. They are:
- **model**: All arguments that are related to the Model - language model, a classifier, optimizer and schedulers, datasets and any other related information

- **trainer**: Any argument to be passed to PyTorch Lightning

In [ ]:
TASK = 'ChemProt'
RE_DATA_DIR = '/content/drive/MyDrive/Life_Singularity/RE'
RE_WORK_DIR = '/content/drive/MyDrive/Life_Singularity/RE/WORK_DIR'
RE_MODEL_CONFIG = 'text_classification_config.yaml'

In [ ]:
# download the model's configuration file 
RE_MODEL_CONFIG = 'text_classification_config.yaml'
RE_config_dir = RE_WORK_DIR + '/configs/'
os.makedirs(RE_config_dir, exist_ok=True)
if not os.path.exists(RE_config_dir + RE_MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/text_classification/conf/' + RE_MODEL_CONFIG, RE_config_dir)
else:
    print ('config file is already exists')

config file is already exists


In [ ]:
# this line will print the entire config of the model
RE_config_path = f'{RE_WORK_DIR}/configs/{RE_MODEL_CONFIG}'
print(RE_config_path)
RE_config = OmegaConf.load(RE_config_path)

/content/drive/MyDrive/Life_Singularity/RE/WORK_DIR/configs/text_classification_config.yaml


In [ ]:
RE_config.model.train_ds.file_path = os.path.join(RE_DATA_DIR, 'train.tsv')
RE_config.model.validation_ds.file_path = os.path.join(RE_DATA_DIR, 'dev.tsv')
RE_config.model.task_name = 'chemprot'
# Note: these are small batch-sizes - increase as appropriate to available GPU capacity
RE_config.model.train_ds.batch_size=8
RE_config.model.validation_ds.batch_size=8
RE_config.model.dataset.num_classes=6
RE_config.trainer.max_epochs=3

In [ ]:
print(OmegaConf.to_yaml(RE_config))

trainer:
  devices: 1
  num_nodes: 1
  max_epochs: 3
  max_steps: -1
  accumulate_grad_batches: 1
  gradient_clip_val: 0.0
  precision: 32
  accelerator: gpu
  log_every_n_steps: 1
  val_check_interval: 1.0
  resume_from_checkpoint: null
  num_sanity_val_steps: 0
  enable_checkpointing: false
  logger: false
model:
  nemo_path: text_classification_model.nemo
  tokenizer:
    tokenizer_name: ${model.language_model.pretrained_model_name}
    vocab_file: null
    tokenizer_model: null
    special_tokens: null
  language_model:
    pretrained_model_name: bert-base-uncased
    lm_checkpoint: null
    config_file: null
    config: null
  classifier_head:
    num_output_layers: 2
    fc_dropout: 0.1
  class_labels:
    class_labels_file: null
  dataset:
    num_classes: 6
    do_lower_case: false
    max_seq_length: 256
    class_balancing: null
    use_cache: false
  train_ds:
    file_path: /content/drive/MyDrive/Life_Singularity/RE/train.tsv
    batch_size: 8
    shuffle: true
    num_samp

In [ ]:
RE_config.model.task_name = TASK
RE_config.model.output_dir = RE_WORK_DIR
RE_config.model.dataset.data_dir = RE_DATA_DIR

In [ ]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(RE_config.trainer))

Trainer config - 

devices: 1
num_nodes: 1
max_epochs: 3
max_steps: -1
accumulate_grad_batches: 1
gradient_clip_val: 0.0
precision: 32
accelerator: gpu
log_every_n_steps: 1
val_check_interval: 1.0
resume_from_checkpoint: null
num_sanity_val_steps: 0
enable_checkpointing: false
logger: false



In [ ]:
# lets modify some trainer configs
# checks if we have GPU available and uses it
RE_config.trainer.devices = 1
RE_config.trainer.accelerator = accelerator

# for PyTorch Native AMP set precision=16
RE_config.trainer.precision = 16 if torch.cuda.is_available() else 32

# remove distributed training flags
RE_config.trainer.strategy = None

RE_trainer = pl.Trainer(**RE_config.trainer)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


In [ ]:
RE_model = nemo_nlp.models.TextClassificationModel.restore_from(restore_path=os.path.join('/content/drive/MyDrive/Life_Singularity/Multiple_Test_Extraction/RE/MODEL', "nemo_re.nemo"), trainer = RE_trainer)

[NeMo W 2022-12-03 16:39:55 lm_utils:80] biomegatron345m_biovocab_30k_uncased is not in get_pretrained_lm_models_list(include_external=False), will be using AutoModel from HuggingFace.


[NeMo I 2022-12-03 16:39:55 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/biomegatron345m_biovocab_30k_uncased/versions/1/files/BioMegatron345m-biovocab-30k-uncased.nemo to /root/.cache/torch/NeMo/NeMo_1.12.0/BioMegatron345m-biovocab-30k-uncased/885893c2477bc7f57b6706bdd8225da0/BioMegatron345m-biovocab-30k-uncased.nemo
[NeMo I 2022-12-03 16:40:23 common:910] Instantiating model from pre-trained checkpoint
[NeMo I 2022-12-03 16:40:28 megatron_init:204] Rank 0 has data parallel group: [0]
[NeMo I 2022-12-03 16:40:28 megatron_init:207] All data parallel group ranks: [[0]]
[NeMo I 2022-12-03 16:40:28 megatron_init:208] Ranks 0 has data parallel rank: 0
[NeMo I 2022-12-03 16:40:28 megatron_init:216] Rank 0 has model parallel group: [0]
[NeMo I 2022-12-03 16:40:28 megatron_init:217] All model parallel group ranks: [[0]]
[NeMo I 2022-12-03 16:40:28 megatron_init:227] Rank 0 has tensor model parallel group: [0]
[NeMo I 2022-12-03 16:40:28 megatron_init:231] All t

[NeMo W 2022-12-03 16:40:28 modelPT:217] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.


[NeMo I 2022-12-03 16:40:28 tokenizer_utils:204] Getting Megatron tokenizer for pretrained model name: megatron-bert-345m-uncased, custom vocab file: /tmp/tmpx8c3hbzn/efa836622369436fa6aa99a1b658adde_pubmed_merged-all-uncased.vocab.txt, and merges file: None
[NeMo I 2022-12-03 16:40:28 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-large-uncased, vocab_file: /tmp/tmpx8c3hbzn/efa836622369436fa6aa99a1b658adde_pubmed_merged-all-uncased.vocab.txt, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.


...

[NeMo I 2022-12-03 16:41:03 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.12.0/BioMegatron345m-biovocab-30k-uncased/885893c2477bc7f57b6706bdd8225da0/BioMegatron345m-biovocab-30k-uncased.nemo.
[NeMo I 2022-12-03 16:41:03 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.12.0/BioMegatron345m-biovocab-30k-uncased/885893c2477bc7f57b6706bdd8225da0/BioMegatron345m-biovocab-30k-uncased.nemo
[NeMo I 2022-12-03 16:41:03 common:910] Instantiating model from pre-trained checkpoint
[NeMo I 2022-12-03 16:41:10 megatron_init:204] Rank 0 has data parallel group: [0]
[NeMo I 2022-12-03 16:41:10 megatron_init:207] All data parallel group ranks: [[0]]
[NeMo I 2022-12-03 16:41:10 megatron_init:208] Ranks 0 has data parallel rank: 0
[NeMo I 2022-12-03 16:41:10 megatron_init:216] Rank 0 has model parallel group: [0]
[NeMo I 2022-12-03 16:41:10 megatron_init:217] All model parallel group ranks: [[0]]
[NeMo I 2022-12-03 16:41:10 megatron_init:227] Rank 0 has tensor model parallel 

[NeMo W 2022-12-03 16:41:10 modelPT:217] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.


[NeMo I 2022-12-03 16:41:10 tokenizer_utils:204] Getting Megatron tokenizer for pretrained model name: megatron-bert-345m-uncased, custom vocab file: /tmp/tmpd_p_jdyh/efa836622369436fa6aa99a1b658adde_pubmed_merged-all-uncased.vocab.txt, and merges file: None
[NeMo I 2022-12-03 16:41:10 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-large-uncased, vocab_file: /tmp/tmpd_p_jdyh/efa836622369436fa6aa99a1b658adde_pubmed_merged-all-uncased.vocab.txt, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.


[NeMo I 2022-12-03 16:41:12 megatron_base_model:185] Padded vocab_size: 31104, original vocab_size: 31077, dummy tokens: 27.
[NeMo I 2022-12-03 16:41:13 save_restore_connector:243] Model MegatronBertModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.12.0/BioMegatron345m-biovocab-30k-uncased/885893c2477bc7f57b6706bdd8225da0/BioMegatron345m-biovocab-30k-uncased.nemo.
[NeMo I 2022-12-03 16:41:13 nlp_model:174] Registering MegatronBERT model config for biomegatron345m_biovocab_30k_uncased is not yet supported.                         Please override this method if needed.
[NeMo I 2022-12-03 16:41:20 save_restore_connector:243] Model TextClassificationModel was successfully restored from /content/drive/MyDrive/Life_Singularity/Multiple_Test_Extraction/RE/MODEL/nemo_re.nemo.


In [ ]:
RE_model.half()

TextClassificationModel(
  (loss): CrossEntropyLoss()
  (bert_model): MegatronBertModel(
    (model): BertModel(
      (language_model): TransformerLanguageModel(
        (embedding): Embedding(
          (word_embeddings): VocabParallelEmbedding()
          (position_embeddings): Embedding(512, 1024)
          (tokentype_embeddings): Embedding(2, 1024)
          (embedding_dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): ParallelTransformer(
          (layers): ModuleList(
            (0): ParallelTransformerLayer(
              (input_layernorm): MixedFusedLayerNorm(torch.Size([1024]), eps=1e-05, elementwise_affine=True)
              (self_attention): ParallelAttention(
                (query_key_value): ColumnLinear()
                (core_attention): CoreAttention(
                  (scale_mask_softmax): FusedScaleMaskSoftmax()
                  (attention_dropout): Dropout(p=0.1, inplace=False)
                )
                (dense): RowParallelLinear()
   

# Organize NER and RE Output


In [ ]:
def merge(queries, labels, B_label, I_label):
  '''
     merge entity chunck in the queries list and labels list
    eg. queries: ['her', 'creatinine', 'was', '1.7', 'on', 'discharge']
        labels: [5, 7, 0, 18, 4, 14]
        user merge() function to merge 'B_Date' and 'I_Date' inr the labels list and corresponding tokens in the queries list:
        merged_queries: ['her', 'creatinine', 'was', '1.7', 'on discharge']
        merged_labels: [5, 7, 0, 18, 4']
   
    :param queries: a list of tokens
    :param labels:  a list of labels
    :param B_label:the class number of B_label need to be merged 
    :param I_label:the class number of I_label need to be merged 
    :return: queries list and labels list after merge
  '''
  for idx in range(len(queries)-1):
    if labels[idx] == B_label and labels[idx+1] == I_label:
      queries[idx] = queries[idx] + " " + queries[idx+1]
      if idx + 2 < len(queries) and labels[idx+2] == I_label:
        queries[idx] = queries[idx] + " " + queries[idx+2]

  for ele,l in zip(queries[:],labels[:]):
    if l == I_label:
      queries.remove(ele)
      labels.remove(l)
  return labels, queries

In [ ]:
from datetime import datetime

def calculateAge(birthday, admission_day):   
  '''
     calcute the age of one patient by his birthday and the admission_date
   
    :param birthday: date of birth, datetime
    :param admission_day:  date of admission, datetime
    :return: age
  '''
  born = datetime.strptime(birthday, "%Y-%m-%d")
  admission = datetime.strptime(admission_day, "%Y-%m-%d")
  print((admission.month, admission.day) < (born.month, born.day))
  return admission.year - born.year - ((admission.month, admission.day) < (born.month, born.day))

In [ ]:
def insert(sql, table_name):
  '''
     insert one record to the database, and return the index of the new inserted record
   
    :param sql: ad string of sql insertion statement
    :param table_name:  the table will be updated
    :return: the index of the new inserted record
  '''
  conn.execute(sql);
  cursor = conn.execute(f"select seq from sqlite_sequence where name='{table_name}';")
  insert_id = cursor.fetchone()[0]
  conn.commit()
  return insert_id

In [ ]:
def genSql(dict, table_name):
  '''
     generate sql insertion statement
   
    :param dict: a dictionary whose key is column names of the table and value is values need to be inserted
    :param table_name:  the table will be updated
    :return: sql insertion statement string
  '''
  string = f"INSERT INTO {table_name} ("
  val_str = "VALUES ("
  keys = list(dict.keys())
  for key in keys:
    string += f"{key}, "
    if key == "begin_offset" or key == "end_offset":
      val_str += f"{dict[key]}, "
    else:
      val_str += f"'{dict[key]}', "
  string = string[:-2]+")"
  val_str = val_str[:-2]+")"
  return string + " " + val_str +";"

In [ ]:
def standardTestName(text):
  '''
     uniform all test name abbrevations to a standard test name
   
    :param text: the original test name ectracted from clinical note
    :return: the standard test name
  '''
  stan_text_name = ""
  if text in ["cr", "creat", "creatinine", "crea", "crn", "ctn", "cre", "crt", "cnn", "crtn"]:
    stan_text_name = "Creatinine"
  elif text in ["alb", "albumin"]:
    stan_text_name = "Albumin"
  elif text in ["bicarbonate", "hco3", "bicarb", "total co2"]:
    stan_text_name = "Bicarbonate"
  elif text in ["totbili", "total Bilirubin"]:
    stan_text_name = "Bilirubin, Total"
  elif text == "crp":
    stan_text_name = "C-Reactive Prot"
  elif text in ["chloride", "cl"]:
    stan_text_name = "Chloride"
  elif text in ["hemoglobin", "hgb"]:
    stan_text_name = "Hemoglobin"
  elif text in ["hematocrit", "hct"]:
    stan_text_name = "Hematocrit"
  elif text in ["ld(ldh)", "ldh", "ld", "lactate", "lactate dehydrogenase"]:
    stan_text_name = "Lactate Dehydro"
  elif text == "mch":
     stan_text_name = "MCH"
  elif text == "mchc":
    stan_text_name = "MCHC"
  elif text == "mcv":
    stan_text_name = "MCV"
  elif text in ["plt ct", "plt count", "platelet count"]:
    stan_text_name = "Platelet Count"
  elif text in ["phosphate", "phos"]:
    stan_text_name = "Phosphate"
  elif text in ["k", "potassium"]:
    stan_text_name = "Potassium"
  elif text == "rdw":
    stan_text_name = "RDW"
  elif text in ["rbc", "red blood cells"]:
    stan_text_name = "Red Blood Cells"
  elif text in ["sodium", "na"]:
    stan_text_name = "Sodium"
  elif text in ["urean", "bun", "urea nitrogen"]:
    stan_text_name = "Urea Nitrogen"
  return  stan_text_name

In [ ]:
ner_Labels_mapping = {'O': 0, 'B_DOA': 1, 'B_DOB': 2, 'B_DOC': 3, 'B_Date': 4, 'B_Heart_Disease': 5, 'B_SEX': 6, 'B_Test_Name': 7, 'B_Time': 8, 'I_DOA': 9, 'I_DOB': 10, 'I_DOC': 11, 'I_Date': 12, 'I_Heart_Disease': 13, 'I_SEX': 14, 'I_Test_Name': 15, 'PER': 16, 'Test_Unit': 17, 'Test_Value': 18}

In [ ]:
def organize((queries, all_preds, adm_id, admDate, discDate):
  '''
    store entity info and it relation to database
   
    :param queries: a list of tokens
    :param all_preds: the prediced entity labels for tokens in the queries list by NER model
    :param adm_id: the admission id of notes
    :param adm_date: the admission date of notes
    :param discDate: the discharge date of notes
  '''
  if len(tokens) > 2 and date_pattern.match(tokens[0]) and time_pattern.match(tokens[1]):
    test_date = tokens[0]
    test_time = tokens[1]
    tests = []
  for i, token in enumerate(tokens):
    if i < len(tokens) - 1 and token in test_list:
      test = token
      value = tokens[i+1]
      tests.append((test, value))
    elif i < len(tokens) - 1 and tokens[i] == "heart" and tokens[i+1] == "failure":
      chf_status = 1
    elif tokens[i] == "chf":
      chf_status = 1
  if len(tests) > 0:
    conn.execute(f"INSERT OR IGNORE INTO Patient (subject_id, date_of_birth, gender) VALUES ('{pid}','{birthday}','{gender}');")
    insert_pid = conn.execute(f"select patient_id from Patient where subject_id='{pid}';").fetchone()[0]
    insert_aid = insert(f"INSERT OR IGNORE INTO Admission (Hadm, admission_date, discharge_date) VALUES ('{admId}', '{admission_date}', '{discharge_date}');", "Admission")
    conn.execute(f"INSERT OR IGNORE INTO Patient_Admission (patient_id, admission_id) VALUES ({insert_pid}, {insert_aid});")
    conn.commit()
    for t in tests:
      test = t[0]
      value = t[1]
      stan_test_name = standardTestName(test)
      test_id  = insert(f"INSERT INTO Test (test_date, test_time, test_value, test_type) VALUES ('{test_date}', '{test_time}', '{value}', '{stan_test_name}');", "Test")
      conn.execute(f"INSERT INTO Admission_Test(admission_id, test_id) VALUES ({insert_aid}, {test_id});") 
  conn.execute(f"UPDATE Admission SET chf_diagnosis = {chf_status} WHERE admission_id = '{insert_aid}'")

In [ ]:
import re
import copy
value_pattern = re.compile(r"^(\d+\.{0,1}\d{1,2})$")

def organize(queries, all_preds, adm_id, admDate, discDate):
  '''
    store entity info and it relation to database
   
    :param queries: a list of tokens
    :param all_preds: the prediced entity labels for tokens in the queries list by NER model
    :param adm_id: the admission id of notes
    :param adm_date: the admission date of notes
    :param discDate: the discharge date of notes
  '''
  start_idx, end_idx, BeginOffset, EndOffset = 0, 0, 0, 0
  chf_status = 0
  for query in queries:
    end_idx += len(query)
    preds = all_preds[start_idx:end_idx]
    start_idx = end_idx
    date_idx, test_idx, value_idx, time_idx, unit_idx, dig_idx = {}, {}, {}, {}, {}, {}
    preds_merge, query_merge = merge(query, preds, 4, 12) #merge B_Date and I_Date
    preds_merge, query_merge = merge(query, preds, 7, 15) #merge B_Test_Name and I_Test_Name
    preds_merge, query_merge = merge(query, preds, 5, 13) #merge B_Heart_Disease and I_Heart_Disease
    date_test_val = []
    for j, word in enumerate(query_merge):
      EndOffset = BeginOffset+len(word)
      if preds_merge[j] == 4:
          if "admission" in word:
              date = admDate
          elif "discharge" in word:
              date = discDate
          else:
              date = word
          date_idx[j] = {"begin_offset":BeginOffset, "end_offset":EndOffset, "text":word, "date":date, "entity_type":"B_Date"}
      elif preds_merge[j] == 7:
          stan_test_name = standardTestName(word)
          if stan_test_name != '':
            test_idx[j] = {"begin_offset":BeginOffset, "end_offset":EndOffset, "text":word, "entity_type":"Test_Name", "test_type": stan_test_name}
      elif preds_merge[j] == 8:
          time_idx[j] = {"begin_offset":BeginOffset, "end_offset":EndOffset, "text":word, "entity_type":"B_Time"}
      elif preds_merge[j] == 18 and value_pattern.match(word):
          value_idx[j] = {"begin_offset":BeginOffset, "end_offset":EndOffset, "text":word, "entity_type":"Test_Value"}
      elif preds_merge[j] == 17:
          unit_idx[j] = {"begin_offset":BeginOffset, "end_offset":EndOffset, "text":word, "entity_type":"Test_Unit"}
      elif preds_merge[j] == 5:
          chf_status = 1

      BeginOffset = EndOffset + 1
    BeginOffset += 1
    
    if len(date_idx) > 0 and len(test_idx) > 0 and len(value_idx) > 0:
      for didx in date_idx.keys():
        sen = copy.deepcopy(query_merge)
        sen[didx] = "B_Date"
        for tidx in test_idx.keys():
          sen[tidx] = "B_Test_Name"
          date_test_rel = RE_model.classifytext([" ".join(sen)])
          if date_test_rel[0] == 1:
            sen[didx] = query_merge[didx]
            for vidx in value_idx.keys():
              sen[vidx] = "Test_Value"
              test_val_rel = RE_model.classifytext([" ".join(sen)])
              if test_val_rel[0] == 0:
                date_test_val.append([didx, tidx, vidx,-1])
              sen[vidx] = query_merge[vidx]
          sen[tidx] = query_merge[tidx]
    
    if len(date_test_val) > 0:
      if len(time_idx) > 0:
        for midx in time_idx.keys():
          sen = copy.deepcopy(query_merge)
          sen[midx] = "B_Time"
          for i, idx in enumerate(date_test_val):
            sen[idx[0]] = "B_Date"
            date_time_rel = RE_model.classifytext([" ".join(sen)])
            if date_time_rel[0] == 3:
              sen[idx[0]] = query_merge[idx[0]]
              sen[idx[1]] = "B_Test_Name"
              time_test_rel = RE_model.classifytext([" ".join(sen)])
              if time_test_rel[0] == 4:
                date_test_val[i][3] = midx
            sen[idx[0]] = query_merge[idx[0]]
            sen[idx[1]] = query_merge[idx[1]]
              
      if len(unit_idx) > 0:
        for uidx in unit_idx.keys():
          sen = copy.deepcopy(query_merge)
          sen[uidx] = "Test_Unit"
          for i, idx in enumerate(date_test_val): 
            sen[idx[2]] = "Test_Value"
            val_unit_rel = RE_model.classifytext([" ".join(sen)])
            if val_unit_rel[0] == 5:
              date_test_val[i].append(uidx)
            sen[idx[2]] = query_merge[idx[2]]
          
    for idx in date_test_val: 
      test_date = date_idx[idx[0]]["date"]
      test_value = value_idx[idx[2]]["text"]
      test_type = test_idx[idx[1]]["test_type"]
      if idx[3] != -1:
        test_time = time_idx[idx[3]]["text"]
      else:
        test_time = "00:00AM"
      test_id  = insert(f"INSERT INTO Test (test_date, test_time, test_value, test_type) VALUES ('{test_date}', '{test_time}', '{value}', '{test_type}');", "Test")
      conn.execute(f"INSERT INTO Admission_Test(admission_id, test_id) VALUES ({adm_id}, {test_id});") 
    conn.execute(f"UPDATE Admission SET chf_diagnosis = {chf_status} WHERE admission_id = {adm_id}")
    conn.commit()
    

In [ ]:
per_pattern = re.compile(r"(\d*,\d*,\d*,\d{4}-\d{1,2}-\d{1,2})")
date_pattern = re.compile(r"\d{4}-\d{1,2}-\d{1,2}")

for j,note in enumerate(process_data):
  pid, admId, admission_date, discharge_date, birthday, gender= "", "","","","",""
  line = (note[0]+" "+note[1]).split()
  for i, token in enumerate(line):
    if per_pattern.match(token):
      (r, pid, admId, cd) = token.split(',')
    elif token == "admission" and line[i+1] == "date" and line[i+2] == ":":
      if len(line) and date_pattern.match(line[i+3]):
        admission_date = line[i+3]
    elif token == "discharge" and line[i+1] == "date" and line[i+2] == ":" and len(line) > i+3 and date_pattern.match(line[i+3]):
      discharge_date = line[i+3]
  birth = note[2].split()
  if len(birth) > 3 and birth[2] == "birth" and date_pattern.match(birth[-1]):
    birthday = birth[-1]
  else:
    birthday = "Unknown"
  gen = note[3].split()
  if gen[0] == "sex":
    if gen[-1] == "f":
      gender = "Female"
    elif gen[-1] == "m":
      gender = "Male"
    else:
      gender = "Unknown"
  conn.execute(f"INSERT OR IGNORE INTO Patient (subject_id, date_of_birth, gender) VALUES ('{pid}','{birthday}','{gender}');")
  insert_pid = conn.execute(f"select patient_id from Patient where subject_id='{pid}';").fetchone()[0]
  conn.commit()
  insert_aid = insert(f"INSERT INTO Admission (Hadm, admission_date, discharge_date) VALUES ('{admId}', '{admission_date}', '{discharge_date}');", "Admission")
  conn.execute(f"INSERT INTO Patient_Admission (patient_id, admission_id) VALUES ({insert_pid}, {insert_aid});")
  conn.commit()
  all_preds = NER_model._infer(note, 1)
  queries = [q.strip().split() for q in note]
  organize(queries, all_preds, insert_aid, admission_date, discharge_date)